# COVID-19 Data Analysis
📊 Analisis tren, penyebaran, dan dampak pandemi COVID-19 di Indonesia.  
🔧 Tools: Python (Pandas, Seaborn), Tableau  

## Insight Utama
- Indonesia mengalami puncak kasus pada ...
- Provinsi X memiliki tingkat kematian tertinggi ...

## Struktur Proyek
- data/: Dataset mentah
- notebooks/: Notebook analisis & prediksi
- dashboard/: Visualisasi interaktif

# **Import Library**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
import plotly as ply
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"
%matplotlib inline

# **Load Data**

In [ ]:
df = pd.read_csv('covid_19_indonesia_time_series_all.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(df.isnull().sum().sort_values(ascending=False))

# **Drop Table not using**

## **Table yang digunakan**
- Date
- Location
- Province (jika fokus ke provinsi)
- Country
- Cases
- Total Cases
- Deaths
- Total Deaths
- Recovered
- Total Recovered
- Active Cases
- Total Active Cases
- Population
- Population Density
- Area (km2)
- Case Recovered Rate
- Case Fatality Rate

In [ ]:
# Delete original date column
cleaned_data = df.drop(columns=['City or Regency','Special Status','Growth Factor of Deaths','Growth Factor of Cases',
                       'Total Rural Villages','Island','Time Zone','Total Urban Villages','Total Cities','Location ISO Code',
                       'Total Districts','Total Regencies','Total Regencies','Continent'])

# **Preprocessing Data**

1. mode() akan mengembalikan nilai yang paling sering muncul di kolom tersebut.
1. [0] dipakai karena mode() bisa mengembalikan lebih dari satu nilai kalau ada beberapa provinsi dengan jumlah kemunculan sama banyak, tapi biasanya cukup ambil yang pertama.

In [ ]:
# Langkah 1: Dapatkan nilai modus untuk Province
most_common_province = cleaned_data['Province'].mode()[0]  # Mengambil nilai modus dari kolom 'Province'
print(f"Provinsi yang paling sering muncul: {most_common_province}")

In [ ]:
# Sesudah (Paling Benar dan Idiomatik di Pandas):
province_stats_filtered = cleaned_data[cleaned_data['Location'] != 'Indonesia'].copy().reset_index(drop=True)

In [ ]:
print(province_stats_filtered.isnull().sum().sort_values(ascending=False))

In [ ]:
province_stats_filtered.head()

Untuk memilih beberapa kolom sekaligus, Anda harus menggunakan list ([]), bukan tuple atau multiple arguments.

Penjelasan Perubahan:
1. Ganti tanda kurung tunggal ('Cases', ...) menjadi double brackets [['Cases', ...]] (Ini adalah cara pandas untuk memilih subset kolom)
1. Hasilnya akan menampilkan tabel dengan: Kolom: Cases, Deaths, Recovered, Active Cases Baris: min (nilai terkecil) dan max (nilai terbesar)

In [ ]:
cek_min_max_values = province_stats_filtered[['Cases','Deaths', 'Recovered','Active Cases']].agg(['min', 'max'])
print(cek_min_max_values)

In [ ]:
# Convert Dtype "Date" dari object -> Dtype datetime
province_stats_filtered['Date'] = pd.to_datetime(province_stats_filtered['Date'])

In [ ]:
province_stats_filtered.info()

# **Exploratory Data Analysis (EDA)**

## **Daily Trends Covind-19 (Cases, Deaths, Recovered, Active Cases)**

In [ ]:
Cases = ['Cases','Deaths', 'Recovered','Active Cases']

In [ ]:
df_daily_trends = province_stats_filtered.copy()

In [ ]:
df_daily_trends.info()

In [ ]:
df_daily_trends.set_index('Date', inplace=True)
axes = df_daily_trends[Cases].plot(marker=".", alpha=0.5, linestyle='None', figsize=(16, 10), subplots=True)

fig = axes[0].figure  # Ambil figure dari subplot pertama
fig.suptitle('COVID-19 Daily Trends Per Case Type')

for i in axes:
    i.set_xlabel('Date')
    i.set_ylabel('Cases')

### **Analisis COVID-19 Daily Trends Per Case Type**

<!DOCTYPE html>
<html>
    <body>
        <p style="text-align:justify ; font-size:90% ; margin: 30px ;">
        Grafik menunjukkan pola gelombang yang sangat jelas untuk Cases, Deaths, dan Recovered, ini menunjukkan puncak gelombang pandemi yang signifikan. Puncak pertama terjadi pada pertengahan tahun 2021, dan pola gelombang menunjukkan puncak yang sangat tinggi untuk Cases dan Deaths pada pertengahan tahun 2021 (sekitar Juli-Agustus). Kemungkinan besar adalah gelombang Delta yang melanda Indonesia. Setelah gelombang kasus baru, Recovered menunjukkan lonjakan yang signifikan. Ini menunjukkan kapasitas pemulihan yang tinggi atau skala penyebaran yang luas.  
        <br>Kemudian gelombang kedua terjadi pada awal tahun 2022 (sekitar Februari - Maret). Dimana puncak gelombang kedua relatif signifikan, walaupun tidak setinggi puncak pertama. Kemungkinan besar adalah gelombang Omicron, dimana Puncak Recovered juga mengikuti pola ini. Menjelang akhir periode tren mengalami penurunan (sekitar pertengahan hingga akhir 2022). Jumlah Cases, Deaths, dan Recovered tampak menurun secara signifikan, kembali ke tingkat yang lebih rendah mendekati garis dasar. Ini menunjukkan fase di mana pandemi mungkin sudah lebih terkendali atau tingkat kekebalan populasi sudah meningkat.
        </p">
        <P style="margin-bottom: 0.1; line-height: 0; font-weight: bold;">Berikut merupakan correlation antar metrik cases</P>
        <ul style="text-align:justify ; font-size:90% ; margin: 30px; margin-top:0.0;">
            <li>
            Terdapat korelasi yang jelas antara Cases dan Deaths, setiap lonjakan Cases selalu diikuti oleh lonjakan Deaths beberapa waktu kemudian akan tetapi terdapat lag waktu yang wajar. Ini menunjukkan bahwa, dalam proporsi yang berbeda, peningkatan kasus infeksi mengarah pada peningkatan kematian. Begitupun dengan kasus kesembuhan juga berkorelasi positif dengan Cases infeksi. Dikarenakan saat Cases infeksi mengalami tren lonjakan, maka Cases sembuh juga akan ikut mengalami tren lonjakan. Hal ini menunjukkan bahwa sistem kesehatan menangani sebagian besar kasus dengan baik, akan tetapi ini juga menunjukkan seberapa cepat virus menyebar.
            </li>
            <li>
            Glombang pada grafik Kasus Active Cases menunjukkan fluktuasi yang lebih tajam dan sering bergerak disekitar nol. Tren lonjakkan positif pada Active Cases terjadi saat Cases infeksi melebihi probabilitas Recovered dan Deaths, hal ini menandakan bahwa infeksi Active Cases di lingkungan population mengalamami peningkatan. Sedangkan untuk Tren lonjakkan negatif pada Active Case terjadi ketika probabilitas Recovered dan Deaths melampaui Cases infeksi, artinya jumlah Active Cases dilingkungan population mengalami penurunan. Hal ini diperkuat dengan garis pada Active Cases yang cenderung fluktuatif disekitar angka nol setelah terjadinya gelombang besar. Hal ini menunjukkan bahwa probabilitas population yang baru terinfeksi relatif simetris dengan probabilitas population yang sembuh atau meninggal.  
            </li>
        </ul>
    </body>
</html>

## **COVID-19 Daily Graph with 7-Day Moving Average**

In [ ]:
df_weekly = province_stats_filtered.copy()

# Convert Dtype "Date" dari object -> Dtype datetime
df_weekly['Date'] = pd.to_datetime(df_weekly['Date'])

In [ ]:
df_weekly.info()

In [ ]:
# Urutkan berdasarkan tanggal
df_weekly.sort_values('Date', inplace=True)

# Hitung 7-day moving average untuk masing-masing metrik
df_weekly['Cases MA'] = df_weekly['Cases'].rolling(window=7).mean()
df_weekly['Deaths MA'] = df_weekly['Deaths'].rolling(window=7).mean()
df_weekly['Recovered MA'] = df_weekly['Recovered'].rolling(window=7).mean()
df_weekly['Active Cases MA'] = df_weekly['Active Cases'].rolling(window=7).mean()

In [ ]:
# Plotting
plt.figure(figsize=(14, 6)) # Set ukuran figure

plt.plot(df_weekly['Date'],df_weekly['Cases MA'], label='Cases (7-day MA)', color='red')
plt.plot(df_weekly['Date'],df_weekly['Deaths MA'], label='Deaths (7-day MA)', color='green')
plt.plot(df_weekly['Date'],df_weekly['Recovered MA'], label='Recovered (7-day MA)', color='blue')
plt.plot(df_weekly['Date'],df_weekly['Active Cases MA'], label='Active Cases (7-day MA)', color='orange')

# Konfigurasi plot 7-day moving average
plt.title('COVID-19 Daily Trends with 7-Day Moving Average')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.show()

Dilihat dari Chart **COVID-19 Daily Trends with 7-Day Moving Average** diatas :
*   Dua gelombang besar pandemi tercermin sangat jelas. Periode pertama pada pertengahan 2021 kemungkinan varian delta dan Periode kedua terjadi pada awal 2022 kemungkinan varian omicron.
*   Kasus kematian naik mengikuti pola lonjakan kasus, menunjukkan bahwa meskipun kasus tinggi, tingkat kematian lebih rendah di gelombang kedua.
*   Recovered cukup tinggi tetapi cepat menurun saat cases menurun, efektivitas lockdown, vaksinasi dan lain-lain.
*   Nilai negatif pada Cases Active merupakan indikator positif bahwa pandemi bisa dikendalikan.




## **Comparative Analysis of Covid-19 Daily Trends Using Logarithmic Analysis**

In [ ]:
# Copy dataset yang sudah bersih
cases_vs_deaths = province_stats_filtered.copy()

In [ ]:
# 1. Seleksi kolom yang akan digunakan
used_columns = cases_vs_deaths[['Date','Cases','Deaths','Recovered']]

In [ ]:
# 2. Convert ke long format
df_long = used_columns.melt(
    id_vars='Date',
    var_name='Metric',
    value_name='Value'
)

In [ ]:
df_long

In [ ]:
# Check Min Max row Value In df_long
Check_Min_Max_log = df_long.agg({
    "Value": ["min","max"],
})
print(Check_Min_Max_log)

In [ ]:
# Ganti nilai 0 menjadi 1 pada kolom 'Value' untuk metrik 'Cases' dan 'Deaths'
# Ini adalah pendekatan umum untuk skala log, karena log(0) tidak terdefinisi
# df_long.loc[(df_long['Metric'].isin(['Cases', 'Deaths'])) & (df_long['Value'] == 0), 'Value'] = 1

# Jika ada nilai negatif (ini seharusnya tidak terjadi untuk kasus/kematian kumulatif,
# tetapi bisa terjadi jika data sumber Anda memiliki anomali atau data harian "New Cases" yang dihitung salah),
# Anda bisa menggantinya dengan 1 atau nilai positif kecil.
# df_long.loc[(df_long['Metric'].isin(['Cases', 'Deaths'])) & (df_long['Value'] < 1), 'Value'] = 1

In [ ]:
# 3. Plot
# # Filter outlier (contoh: nilai > 99th percentile)
# q95 = df_long['Value'].quantile(0.95) # quartile 95
# df_clean = df_long[df_long['Value'] <= q95]

# Plot interaktif
fig = px.line(
    df_long,
    x='Date',
    y='Value',
    color='Metric',
    log_y=True, # mengubah nilai 0 menjadi 1 saat akan melakukan plot, tetapi jika terdapat nilai minus(-) maka akan error
    title="Interaktif: Tren Harian Infeksi Kasus & Kematian COVID-19 (Skala Log, 2020–2022)",
    labels={"Value": "Jumlah (Log)", "Date": "Date"},
    template='plotly_dark'
)

# Konfigurasi layout interaktif
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=False), # untuk menampilkan slider, jika true dia ditampilkan jika false sebaliknya
        type="date",
        title_font=dict(size=24), # untuk mengatur ukuran font "Tanggal"
        tickfont=dict(size=10) # Untuk mengatur ukuran font "Bulan dan Tahun"
    ),
    hovermode="x unified",
    legend_title_text='Metric',
    margin=dict(l=20, r=20, t=50, b=10)
)

fig.show()

<!DOCTYPE html>
<!DOCTYPE html>
<html>
<head>
    <style>
        h3{
            text-align:justify ;
            font-weight: bold;
        }
        .styled-text{
            text-align:justify ;
            font-size: 16 px;
            /* color: white; */
            margin: 30px;
            font-family: "Times New Roman";
            margin-bottom: 0.0;
            margin-top: 0.0;
        }
    </style>
</head>
    <body>
        <h3 href="h3">Analisis Tren Harian Infeksi Kasus & Kematian COVID-19 (Skala Log, 2020–2022)</h3>
        <p class="styled-text"> Grafik ini memvisualisasikan infeksi Cases Covid-19, Recovered dan meninggal harian menggunakan skala logaritmik (Sumbu Y) dari 2020 hingga 2022</p>
        <ul style="margin-top: 0;">
            <li class="styled-text"><strong>Gelombang pertama (Sekitar Juli-Agustus 2021) :</strong>Terlihat puncak yang sangat mencolok pada Kasus dan Kematian di pertengahan tahun 2021 (gelombang Delta). Kemudian Recovered menunjukkan tren peningkatan yang signifikan mengikuti pola Kasus dan sering kali dengan angka yang lebih tinggi dari pada puncak Kasus infeksi. Hal ini menggambarkan sistem kesehatan dalam mendokumentasikan pemulihan setelah kenaikan jumlah kasus.</li>
            <li class="styled-text"><strong>Gelombang Kedua (Sekitar Februari-Maret 2022) :</strong>Gelombang ke 2 yang terjadi diawal tahun 2022 (gelombang omnicron) terlihat sangat jelas. Kasus Infeksi covid-19 menunjukkan lonjakan besar dan bahkan Recovered ikut menunjukkan lonjakan besar, yang dimana sampai melampaui Gelombang puncak dari Kasus Infeksi. Namun gelombang Kematian tampak lebih rendah dibandingkan gelombang Delta, ini mengindikasikan bahwa efisiensi pemulihan yang sangat tinggi.</li>
            <li class="styled-text"><strong>Tren Menurun Akhir Periode :</strong> Menjelang akhir periode dataset (pertengahan hingga akhir 2022), semua metric Kasus Infeksi, Recovered dan Deaths menunjukkan penurunan yang signifikan dan konsisten, kembali kepada tingkat terendahnya.</li>
        </ul>
         <h4 style = "font-weight: bold; font-size: 16px; margin: 30px; margin-top: 0.0; margin-bottom: 0.0;">Result: </h4>
         <p class="styled-text"> Melihat hasil visualisasi kasus Recovered terlihat mendominasi pada grafik, pada sebagian periode volume kasus Recovered terlihat lebih tinggi atau setidaknya ekuivalen dengan Kasus infeksi setelah puncak gelombang. Hal ini merupakan indikator positif bahwa sebagian individu yang terinfeksi berhasil sembuh. Proporsi Deaths dari visualisasi grafik, menunjukkan bahwa jumlah kematian daily jauh lebih rendah dibandingkan dengan Kasus Infeksi dan Kasus Sembuh. Lonjakan yang terjadi pada Kasus Infeksi selalu diikuti juga oleh Kasus Recovered, hal ini secara alami mengarah pada peningkatan Kasus Recovered setelah periode pemulihan.</p>
    </body>
</html>

## **Covid-19 Cases Distribution Map Graphic per Province:**

In [ ]:
province_stats = province_stats_filtered.copy()

In [ ]:
province_stats

In [ ]:
# 1. Seleksi kolom yang akan digunakan
# used_columns_stats = province_stats[["Date","Province","Longitude","Latitude"]]

In [ ]:
# 2. Urutkan berdasarkan tanggal dan grupkan per provinsi untuk mendapatkan entri terbaru
# latest_province_data = province_stats.sort_values('Date').groupby('Province').last().reset_index()

In [ ]:
# 3. Pilih kolom-kolom yang ingin dipetakan (sesuai permintaan Anda)
cases_columns = ['Cases', 'Deaths', 'Recovered']

In [ ]:
min_max_value = province_stats.agg({
    'Cases' : ['min', 'max'],
    'Deaths' : ['min', 'max'],
    'Recovered' : ['min', 'max'],
    'Active Cases' : ['min', 'max'],
})

print(min_max_value)

In [ ]:
from urllib.request import urlopen
import json

with open(r"Indonesia_province.json", 'r') as f: # r didepan memberi tahu Python untuk memperlakukan backslash secara literal, bukan sebagai karakter escape.
    geojson_data = json.load(f)

In [ ]:
# 4. Membuat Fungsi untuk Peta
# Kita akan membuat satu peta per jenis kasus agar lebih mudah dibaca.
# Atau, Anda bisa membuat satu peta dengan drop-down untuk memilih jenis kasus.
# color_scale='RdYlGn_r' --> 'RdYlGn_r' merupakan singkatan dari warna Red,Yellow and Green, fungsi_r untuk reverse. Yang artinya posisi warna menjadi ditukar.
def create_case_map(province_stats,case_type = cases_columns, geojson = geojson_data, color_scale='RdYlGn_r', title_suffix=""):
    """
    Membuat peta interaktif untuk jenis kasus tertentu.
    df: DataFrame yang sudah diagregasi per provinsi.
    case_type: String, nama kolom kasus (e.g., 'Cases').
    color_scale: Skema warna Plotly (misalnya 'Reds', 'Greens', 'RdYlGn_r' untuk Merah-Kuning-Hijau terbalik).
                 '_r' berarti reverse, jadi merah=tinggi, hijau=rendah.
    title_suffix: String tambahan untuk judul peta.
    """
    fig = px.scatter_map(province_stats,
                            lat="Latitude",
                            lon="Longitude",
                            size=case_type, # Ukuran titik berdasarkan jumlah kasus
                            color=case_type, # Warna titik berdasarkan jumlah kasus
                            color_continuous_scale=color_scale, # Skema warna
                            size_max=40, # Ukuran maksimal titik
                            zoom=4, # Zoom awal peta
                            hover_name="Province", # Teks yang muncul saat hover
                            hover_data={
                                "Date": "|%Y-%m-%d", # Format tanggal di hover
                                "Province": True,
                                case_type: True,
                                "Cases": True,
                                "Deaths": True,
                                "Recovered": True,
                                "Latitude": False, # Sembunyikan dari hover
                                "Longitude": False # Sembunyikan dari hover
                            },
                            # title=f'\nPeta Sebaran {case_type} per Provinsi {title_suffix}'
                            map_style="open-street-map" # Gaya peta, bisa 'open-street-map', 'carto-positron', dll.
                           )
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        hovermode="x unified",
        coloraxis_colorbar=dict(
            title=case_type # Judul color bar
        ),
        title={
                'text': f"Peta Sebaran {case_type} per Provinsi {title_suffix}", # Teks judul
                'y': 0.98,              # Posisi vertikal judul (0.0 = bawah, 1.0 = atas figur)
                'x': 0.01,               # Posisi horizontal judul (0.0 = kiri, 0.5 = tengah, 1.0 = kanan figur)
                'xanchor': 'left',    # Anchor horizontal (titik referensi 'x')
                'yanchor': 'top'        # Anchor vertikal (titik referensi 'y')
            }
    )
    return fig

# # 5. Tampilkan Peta untuk Setiap Jenis Kasus ---
all_figs = []
for col in cases_columns:
    if col == 'Recovered':
        fig_recovered = create_case_map(province_stats, 'Recovered', color_scale='YlGn', geojson=geojson_data)
        all_figs.append(fig_recovered.show())
    else:
        fig = create_case_map(province_stats, col, color_scale='RdYlGn_r', geojson= geojson_data)
        all_figs.append(fig.show())

# Contoh pemanggilan fungsi untuk recovered dengan skala warna berbeda
# fig_recovered = create_case_map(province_stats, 'Recovered', color_scale='YlGn', geojson=geojson_data, title_suffix='')
# fig_recovered.show()

<!DOCTYPE html>
<!DOCTYPE html>
<html>
<head>
    <style>
        h3{
            text-align:justify ;
            font-weight: bold;
        }
        .styled-text{
            text-align:justify ;
            font-size: 16 px;
            /* color: white; */
            margin: 30px;
            font-family: "Times New Roman";
            margin-bottom: 0.0;
            margin-top: 0.0;
        }
    </style>
</head>
    <body>
        <h3 href="h3">Analisis Peta Sebaran Cases Covid-19, Deaths dan Recovered Daily per Provinsi</h3>
        <p class="styled-text"> Peta sebaran ini memvisualisasikan infeksi Cases Covid-19, Recovered dan Deaths harian dari 2020 hingga 2022</p>
        <ul style="margin-top: 0;">
            <li class="styled-text"><strong>Peta Sebaran Cases per Provinsi :</strong> Warna dan ukuran titik koordinat pada peta menunjukkan jumlah kasus dan keparahan suatu provinsi. Misalnya Jawa Barat yang memiliki ukuran koordinat yang lebih besar dan warna yang cenderung merah dan oranye, hal ini menandakan tingginya dan banyaknya jumlah kasus di Jawa Barat. Terlihat pada peta titik koordinat yang lebih besar ukurannya cenderung terkonsentrasi di pulau Jawa, Sumatera bagian selatan, Sulawesi selatan, Kalimantan timur dan Nusa Tenggara Timur. Hal mengindikasikan wilayah yang padat penduduk</li>
            <li class="styled-text"><strong>Peta Sebaran Deaths per Provinsi :</strong> Pola warna dan ukuran titik koordinat memiliki pola yang ekuivalen dengan Cases, dengan lingkaran besar (jumlah kematian tinggi) terkonsentrasi di wilayah yang sama (Jawa, sebagian Sumatera dan Bali)</li>
            <li class="styled-text"><strong>Peta Sebaran Recovered per Provinsi :</strong> Pola titik koordinat menunjukkan konsistensi, dengan kompabilitas yang tinggi dipulau Jawa, Papua, Sumatera dan Bali.</li>
        </ul>
         <h4 style = "font-weight: bold; font-size: 16px; margin: 30px; margin-top: 0.0; margin-bottom: 0.0;">Result: </h4>
         <p class="styled-text"> Melihat hasil visualisasi kasus Recovered terlihat mendominasi pada grafik, pada sebagian periode volume kasus Recovered terlihat lebih tinggi atau setidaknya ekuivalen dengan Kasus infeksi setelah puncak gelombang. Hal ini merupakan indikator positif bahwa sebagian individu yang terinfeksi berhasil sembuh. Proporsi Deaths dari visualisasi grafik, menunjukkan bahwa jumlah kematian daily jauh lebih rendah dibandingkan dengan Kasus Infeksi dan Kasus Sembuh. Lonjakan yang terjadi pada Kasus Infeksi selalu diikuti juga oleh Kasus Recovered, hal ini secara alami mengarah pada peningkatan Kasus Recovered setelah periode pemulihan.</p>
    </body>
</html>

## **Graph of Cases per 100,000 Population (Cumulative)**

Rumus :
$$ Kasus\:per\:100.000= \left (\frac {\:\:Number\:of\:Cases\:of\:Specific\:Category\:\:} {\:Population\:} \right)\:×\:100.000$$

In [ ]:
df_cases_kumulatif = province_stats_filtered.copy()

In [ ]:
Total_Cases_per_100rb = round((df_cases_kumulatif['Total Cases'] / df_cases_kumulatif['Population']) * 100000, 2)
print(type(Total_Cases_per_100rb))

Total_Recovered_per_100rb = round((df_cases_kumulatif['Total Recovered'] / df_cases_kumulatif['Population']) * 100000, 2)
print(type(Total_Recovered_per_100rb))

Total_ActiveCases_per_100rb = round((df_cases_kumulatif['Total Active Cases'] / df_cases_kumulatif['Population']) * 100000, 2)
print(type(Total_ActiveCases_per_100rb))

In [ ]:
# Using insert() to add a column at position 2 (3rd column)
df_cases_kumulatif.insert(21, "Total Cases per 100rb", Total_Cases_per_100rb, True)
df_cases_kumulatif.insert(23, "Total Recovered per 100rb", Total_Recovered_per_100rb, True)
df_cases_kumulatif.insert(25, "Total Active Cases per 100rb", Total_ActiveCases_per_100rb, True)

In [ ]:
df_cases_kumulatif.info()

In [ ]:
df_cases_kumulatif.head()

In [ ]:
from plotly.subplots import make_subplots

# Agregasi data harian secara nasional atau per provinsi (jika Anda ingin melihat tren kumulatif)
# Untuk line chart, kita biasanya melihat tren dari waktu ke waktu.
# Karena Anda ingin per daerah, mari kita pilih salah satu provinsi saja untuk contoh.
# Atau, jika Anda ingin melihat tren nasional, lakukan agregasi total.

# Contoh: Agregasi per tanggal untuk melihat tren nasional (jika data Anda belum diagregasi per tanggal)
# Anda perlu memutuskan data mana yang akan Anda gunakan untuk line chart.
# Misalnya, jika Anda ingin total harian:
daily_summary = df_cases_kumulatif.groupby('Date').agg({ # .agg() digunakan untuk menerapkan satu atau lebih fungsi agregasi (seperti sum, mean, count, max, min, dll.)
    'Total Cases per 100rb': 'sum', # Sum di gunakan untuk mengagregasi (menggabungkan) data harian dari berbagai lokasi (provinsi/daerah) menjadi satu nilai total untuk setiap tanggal. jadi terhitung untuk nasional
    'Total Recovered per 100rb': 'sum',
    'Total Deaths per 100rb': 'sum',
    'Total Active Cases per 100rb': 'sum',
}).reset_index()

# Jika Anda ingin total kumulatif (sering digunakan untuk line chart tren):
# daily_summary['Total Cases'] = daily_summary['Total Cases per 100rb']
# daily_summary['Total Recovered'] = daily_summary['Total Recovered per 100rb']
# daily_summary['Total Deaths'] = daily_summary['Total Deaths per 100rb']# cumsum() fungsi yang digunakan untuk menghitung jumlah kumulatif dari elemen dalam sebuah array, vektor, atau DataFrame
# daily_summary['Total Active Cases'] = daily_summary['Total Active Cases per 100rb']


# Buat Subplot dengan Sumbu Y Ganda
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Tambahkan 'Total Deaths' ke sumbu Y kanan (sekunder)
fig.add_trace(
    go.Scatter(x=daily_summary['Date'], y=daily_summary['Total Cases per 100rb'], name='Total Cases', mode='lines', line=dict(color='blue')),
    secondary_y=False,
)
# Tambahkan 'Total Recovered' ke sumbu Y kiri atau kanan, tergantung skalanya
# Jika Recovered masih jauh lebih tinggi dari Deaths, mungkin bisa di sumbu Y kiri
# Jika Recovered lebih mirip Deaths, bisa di sumbu Y kanan
fig.add_trace(
    go.Scatter(x=daily_summary['Date'], y=daily_summary['Total Recovered per 100rb'], name='Total Recovered', mode='lines', line=dict(color='green')),
    secondary_y=False, # Sesuaikan ini
)

fig.add_trace(
    go.Scatter(x=daily_summary['Date'], y=daily_summary['Total Deaths per 100rb'], name='Total Deaths', mode='lines', line=dict(color='red')),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=daily_summary['Date'], y=daily_summary['Total Active Cases per 100rb'], name='Total Active Cases', mode='lines', line=dict(color='orange')),
    secondary_y=True,
)
# Tambahkan 'Total Cases' ke sumbu Y kiri (primer)

# Konfigurasi layout
fig.update_layout(
    title_text='Tren Kasus COVID-19 (Kumulatif) Per 100.000 Population',
    hovermode="x unified" # Menampilkan semua info pada hover di tanggal yang sama
)

# Konfigurasi sumbu Y kiri
fig.update_yaxes(title_text="Jumlah Kasus dan Sembuh (Kumulatif)", secondary_y=False)
# Konfigurasi sumbu Y kanan
fig.update_yaxes(title_text="Jumlah Mati dan Active Cases (Kumulatif)", secondary_y=True)

fig.show()

### **Analisis Tren Pada Grafik:**

1. Periode Awal (Awal 2021 - Pertengahan 2021):  
    Ketiga garis (Kasus, Sembuh, Kematian) menunjukkan kenaikan yang relatif landai pada awalnya. Ini mungkin mencerminkan periode awal pandemi atau gelombang pertama yang lebih terkontrol.
    Garis Biru (Total Cases) dan Hijau (Total Recovered) bergerak beriringan, menunjukkan bahwa sebagian besar kasus yang terdeteksi juga berhasil sembuh.
    Garis Merah (Total Deaths) tetap sangat rendah dibandingkan dengan dua garis lainnya, konsisten dengan skala sumbu Y sekunder yang jauh lebih kecil.
2. Periode Peningkatan Pesat (Pertengahan 2021 - Akhir 2021):  
    Terlihat adanya lonjakan sangat tajam pada ketiga garis. Ini menunjukkan adanya gelombang kasus COVID - 19 yang signifikan dalam periode tersebut.
    Total Cases (Biru) menunjukkan kenaikan paling dramatis, mencapai puncaknya (atau stabil pada tingkat tinggi) mendekati 800 ribu. Ini mengindikasikan penyebaran virus yang sangat cepat dan luas.
    Total Recovered (Hijau) mengikuti tren Total Cases dengan jarak yang relatif dekat, namun dengan sedikit jeda. Ini wajar karena kesembuhan terjadi setelah infeksi. Kesenjangan antara biru dan hijau mewakili kasus aktif pada saat itu.
    Total Deaths (Merah) juga menunjukkan peningkatan yang signifikan, meskipun dalam skala yang lebih kecil (mencapai sekitar 15 ribu). Lonjakan kematian ini berkorelasi dengan lonjakan kasus,yang merupakan konsekuensi yang dapat diperkirakan dari gelombang infeksi besar.
3. Periode Stabilisasi/Lanjutan (Akhir 2021 - Awal 2022):  
    Setelah lonjakan tajam, ketiga garis terlihat cenderung mendatar atau sedikit meningkat perlahan. Ini menunjukkan bahwa laju penambahan kasus baru, kesembuhan, dan kematian telah melambat secara signifikan atau mencapai titik stabil.
    Total Cases (Biru) dan Total Recovered (Hijau) hampir menyatu di puncak grafik, menunjukkan bahwa sebagian besar kasus yang terdeteksi telah berhasil sembuh, dan jumlah kasus aktif berkurang.
    Total Deaths (Merah) juga cenderung mendatar, menunjukkan bahwa jumlah kematian baru harian telah menurun atau stabil pada tingkat yang jauh lebih rendah dibandingkan puncak gelombang.
    
    Kesimpulan Umum:  
    Chart ini menggambarkan dampak gelombang besar COVID-19 yang terjadi antara pertengahan hingga akhir tahun 2021, yang menyebabkan peningkatan drastis dalam jumlah kasus, kesembuhan, dan kematian kumulatif. Setelah gelombang tersebut, situasi tampaknya telah terkendali dengan laju penambahan kasus dan kematian yang melambat, serta jumlah kasus sembuh yang mendekati jumlah total kasus.
    Pentingnya menggunakan sumbu Y sekunder untuk "Total Deaths" adalah keputusan yang tepat karena rentang nilainya jauh berbeda dari total kasus dan kesembuhan, memungkinkan visualisasi tren kematian yang lebih jelas tanpa "tenggelam" oleh skala yang lebih besar.


## **Graph of Active Cases, Recoveries and Deaths Ratio (Cumulative)**

Rumus :
$$ Category\:Ratio= \left (\frac {\:\:Cumulative\:Total\:of\:Certain\:National\:Categories\:\:} {Total\:National\:Cumulative\:Cases} \right)\:×\:100\%$$

In [ ]:
df_rasio_cases = df_cases_kumulatif.copy()

In [ ]:
df_rasio_cases.info()

In [ ]:
# Rasio_Kematian = round((df_rasio_cases["Total Deaths"] / df_rasio_cases["Total Cases"]), 2)
# print(type(Rasio_Kematian))

# Rasio_Sembuh = round((df_rasio_cases["Total Recovered"] / df_rasio_cases["Total Cases"]), 2)
# print(type(Rasio_Sembuh))

# Rasio_Kasus_Aktif = round((df_rasio_cases["Total Active Cases"] / df_rasio_cases["Total Cases"]), 2)
# print(type(Rasio_Kasus_Aktif))

In [ ]:
# Using insert() to add a column at position 2 (3rd column)
# df_rasio_cases.insert(25, "Rasio Kematian", Rasio_Kematian, True)
# df_rasio_cases.insert(26, "Rasio Sembuh", Rasio_Sembuh, True)
# df_rasio_cases.insert(27, "Rasio Kasus Aktif", Rasio_Kasus_Aktif, True)

In [ ]:
df_rasio_cases.head()

In [ ]:
# df adalah DataFrame Anda yang berisi data per provinsi per hari
df_national_cumulative = df_rasio_cases.groupby('Date').agg(
    Total_Cases_National=('Total Cases', 'sum'),
    Total_Deaths_National=('Total Deaths', 'sum'),
    Total_Recovered_National=('Total Recovered', 'sum'),
    Total_Active_Cases_National=('Total Active Cases', 'sum')
).reset_index()

In [ ]:
# Check_Min_Max = Feature_Data[['Total Cases per 100rb','Total Deaths per 100rb','Total Recovered per 100rb']].agg(['min,max'])
Check_Min_Max = df_national_cumulative.agg({ # .agg() digunakan untuk menerapkan satu atau lebih fungsi agregasi (seperti sum, mean, count, max, min, dll.).)
    'Total_Cases_National': ['min','max'], # Sum di gunakan untuk mengagregasi (menggabungkan) data harian dari berbagai lokasi (provinsi/daerah) menjadi satu nilai total untuk setiap tanggal. jadi terhitung untuk nasional
    'Total_Deaths_National': ['min','max'],
    'Total_Recovered_National': ['min','max'],
    'Total_Active_Cases_National': ['min','max'],
    # 'Total Active Cases per 100rb': ['min','max'],
})
print(Check_Min_Max)

In [ ]:
# Fungsi pembantu untuk menghindari Divisi oleh Nol
def calculate_ratio(numerator, denominator):
    return (numerator / denominator * 100) if denominator > 0 else 0

df_national_cumulative['Rasio Kematian'] = df_national_cumulative.apply(
    lambda row: calculate_ratio(row['Total_Deaths_National'], row['Total_Cases_National']), axis=1
)
df_national_cumulative['Rasio Sembuh'] = df_national_cumulative.apply(
    lambda row: calculate_ratio(row['Total_Recovered_National'], row['Total_Cases_National']), axis=1
)
df_national_cumulative['Rasio Kasus Aktif'] = df_national_cumulative.apply(
    lambda row: calculate_ratio(row['Total_Active_Cases_National'], row['Total_Cases_National']), axis=1
)

# Pastikan Rasio Kasus Aktif + Rasio Sembuh + Rasio Kematian = 100% (kecuali ada pembulatan)
# Anda juga bisa menambahkan kolom verifikasi:
df_national_cumulative['Total Rasio (%)'] = df_national_cumulative['Rasio Kematian'] + df_national_cumulative['Rasio Sembuh'] + df_national_cumulative['Rasio Kasus Aktif']

# Data ini (df_national_cumulative) yang harus Anda gunakan untuk membuat grafik rasio.

In [ ]:
df_national_cumulative

In [ ]:
# df_comparasion_rasio_cases = df_rasio_cases.groupby('Date').agg({  # .agg() digunakan untuk menerapkan satu atau lebih fungsi agregasi(seperti sum, mean, count, max, min, dll.)
#     'Rasio Kematian': 'sum', # Sum di gunakan untuk mengagregasi (menggabungkan) data harian dari berbagai lokasi (provinsi/daerah) menjadi satu nilai total untuk setiap tanggal. jadi terhitung untuk nasional
#     'Rasio Sembuh': 'sum',
#     'Rasio Kasus Aktif': 'sum',
# }).reset_index()
# print(type(df_comparasion_rasio_cases))

In [ ]:
# Check_Min_Max = Feature_Data[['Total Cases per 100rb','Total Deaths per 100rb','Total Recovered per 100rb']].agg(['min,max'])
Check_Min_Max = df_national_cumulative.agg({ # .agg() digunakan untuk menerapkan satu atau lebih fungsi agregasi (seperti sum, mean, count, max, min, dll.)
    'Rasio Kematian': ['min','max'], # Sum di gunakan untuk mengagregasi (menggabungkan) data harian dari berbagai lokasi (provinsi/daerah) menjadi satu nilai total untuk setiap tanggal. jadi terhitung untuk nasional
    'Rasio Sembuh': ['min','max'],
    'Rasio Kasus Aktif': ['min','max'],
})
print(Check_Min_Max)

In [ ]:
# 1. Seleksi kolom yang akan digunakan
df_used_rasio = df_national_cumulative[['Date','Rasio Kematian','Rasio Sembuh','Rasio Kasus Aktif']]

In [ ]:
# 2. Convert ke long format, ini bertujuan untuk mengelompokkan DataFrame
df_comparasion = df_used_rasio.melt(
    id_vars='Date', # ini berisi Date yang digunakan sebagai
    var_name='Metric', # ini berisi nama kolom yang digunakan
    value_name='Value' # ini berisi nilai
)

In [ ]:
# 3. Membuat Grafik
fig = px.line(df_comparasion,
              x='Date',
              y='Value',
              color='Metric',
              title='Grafik Rasio Kasus Aktif, Sembuh dan Kematian Akumulatif',
              labels={'Value': 'Rasio (%)', 'Date': 'Date'},
              )
fig.update_layout(hovermode="x unified")
fig.show()

<!DOCTYPE html>
<!DOCTYPE html>
<html>
<head>
    <style>
        h3{
            text-align:justify ;
            font-weight: bold;
        }
        .styled-text{
            text-align:justify ;
            font-size: 16 px;
            /* color: white; */
            margin: 30px;
            font-family: "Times New Roman";
            margin-bottom: 0.0;
            margin-top: 0.0;
        }
    </style>
</head>
    <body>
        <h3 href="h3"> Analisis Grafik Rasio Kasus Aktif, Sembuh dan Kematian Akumulatif</h3>
        <p class="styled-text"> Grafik ini menunjukkan epidemiologis dari ketiga rasio dalam memperlihatkan bagaimana infeksi Cases Covid-19 mempengaruhi ketiga rasio yaitu Sembuh, meninggal dan Active Cases.</p>
        <ul style="margin-top: 0;">
            <li class="styled-text"><strong>Tren Rasio Sembuh :</strong> Grafik ini menunjukkan Rasio Sembuh yang stabil dan dominan pada tingkat yang tinggi, mendekati 90%-95% setelah fluktuasi di masa-masa awal pandemi (Maret-April 2020). Ini mengindikasikan bahwa sebagian besar individu yang terinfeksi COVID-19 di Indonesia berhasil pulih. Tingginya rasio kesembuhan ini adalah indikator positif tentang efektivitas penanganan kasus dan kapasitas sistem kesehatan untuk mengelola sebagian besar infeksi.</li>
            <li class="styled-text"><strong>Tren Rasio Active Cases :</strong> Setelah fase fluktuasi awal, Rasio Active Cases menunjukkan tren yang stabil pada tingkat yang sangat rendah mendekati 5% dan di sebagian periode kurang dari 5%. Hal ini menandakan bahwa proporsi Active Cases (dalam masa sakit atau isolasi) relatif kecil dengan total cases yang pernah ada, nilai yang rendah ini mencerminkan efektivitas dalam mengelola kasus dan mitigasi penyebaran lebih lanjut dari Active Cases</li>
            <li class="styled-text"><strong>Tren Rasio Meninggal :</strong> Setelah melewati fluktuasi periode awal, Rasio Meninggal menunjukkan tren yang konsisten sangat rendah dibawah 5%, bahkan mendekati 2%-3%. Hal ini mengindikasikan bahwa meskipun covid-19 penyakit yang serius, proporsi kasus yang berujung menjadi kematian di indonesia relatif kecil dibandingkan dengan total kasus yang terinfeksi. Tingkat ini sejalan dengan data global setelah gelombang pandewi awal dan menunjukkan kapasitas penanganan medis yang memadai.</li>
        </ul>
         <h4 style = "font-weight: bold; font-size: 16px; margin: 30px; margin-bottom: 0.0;">Result: </h4>
         <p class="styled-text"> Hasil ini menunjukkan bahwa mayoritas kasus covid-19 di indonesia berakhir dengan kesembuhan, yang dimana proporsi kasus aktif dan kematian relatif kecil dibandingkan dengan total cases. Tren ini memberikan gambaran tentang dampak kumulatif pandemi di indonesia dari perspektif rasio hasil kasus, dalam memfokuskan upaya pemulihan yang efektif.</p>
    </body>
</html>

## **Covid-19 Risk Zone Distribution Map Per Province in Indonesia**

In [ ]:
df_zona_risiko = df_rasio_cases.copy()

In [ ]:
df_zona_risiko.info()

In [ ]:
# Check_Min_Max = Feature_Data[['Total Cases per 100rb','Total Deaths per 100rb','Total Recovered per 100rb']].agg(['min,max'])
Check_Min_Max = df_zona_risiko.agg({ # .agg() digunakan untuk menerapkan satu atau lebih fungsi agregasi (seperti sum, mean, count, max, min, dll.)
    'Total Cases per 100rb': ['min','max'], # Sum di gunakan untuk mengagregasi (menggabungkan) data harian dari berbagai lokasi (provinsi/daerah) menjadi satu nilai total untuk setiap tanggal. jadi terhitung untuk nasional
    'Total Recovered per 100rb': ['min','max'],
    'Total Active Cases per 100rb': ['min','max'],
    'Total Deaths per 100rb': ['min','max'],
})
print(Check_Min_Max)

In [ ]:
# Check_Min_Max = Feature_Data[['Total Cases per 100rb','Total Deaths per 100rb','Total Recovered per 100rb']].agg(['min,max'])
Check_Min_Max_Province = df_zona_risiko.groupby("Province").agg({
    'Total Cases per 100rb': ['min','max'], # Sum di gunakan untuk mengagregasi (menggabungkan) data harian dari berbagai lokasi (provinsi/daerah) menjadi satu nilai total untuk setiap tanggal. jadi terhitung untuk nasional
    'Total Recovered per 100rb': ['min','max'],
    'Total Active Cases per 100rb': ['min','max'],
    'Total Deaths per 100rb': ['min','max'],
})

# Jika ingin mengatur properti tampilan (misalnya warna, lebar kolom, dll), kamu harus menggunakan .style terlebih dahulu sebelum memanggil .set_properties().
display(Check_Min_Max_Province.map('{:.2f}'.format).style.set_table_styles([{'selector': 'th:td', 'props': [('text-align', 'justify')]}])) # menggunakan fungsi display(df) atau hanya mengetik df di sel kode. Ini akan menampilkan DataFrame sebagai tabel HTML

In [ ]:
# Mengecek nilai Quartile dan Melakukan uji coba menggunakan Persentil, hal ini dilakukan guna mempersiapkan THRESHOLD untuk Colums "Zona Risiko"

# Tampilkan statistik deskriptif untuk kolom-kolom yang relevan
print(df_zona_risiko[['Total Cases per 100rb', 'Total Recovered per 100rb', 'Total Deaths per 100rb']].describe())

# Atau lihat histogram untuk visualisasi distribusi
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.histplot(df_zona_risiko['Total Cases per 100rb'], bins=50)
plt.title('Distribusi Total Cases per 100rb')
plt.subplot(1, 3, 2)
sns.histplot(df_zona_risiko['Total Recovered per 100rb'], bins=50)
plt.title('Distribusi Total Recovered per 100rb')
plt.subplot(1, 3, 3)
sns.histplot(df_zona_risiko['Total Deaths per 100rb'], bins=50)
plt.title('Distribusi Total Deaths per 100rb')
plt.tight_layout()
plt.show()

# Lihat nilai persentil untuk mendapatkan gambaran yang lebih baik
print("Percentiles for Total Cases per 100rb:")
print(df_zona_risiko['Total Cases per 100rb'].quantile([0.25, 0.5, 0.75, 0.90, 0.95, 0.99]))
print("\nPercentiles for Total Recovered per 100rb:")
print(df_zona_risiko['Total Recovered per 100rb'].quantile([0.25, 0.5, 0.75, 0.90, 0.95, 0.99]))
print("\nPercentiles for Total Deaths per 100rb:")
print(df_zona_risiko['Total Deaths per 100rb'].quantile([0.25, 0.5, 0.75, 0.90, 0.95, 0.99]))

Berikut merupakan Rumus perhitungan THRESHOLD untuk setiap metrik menggunakan P90 untuk kondisi HIGH dan P50 untuk kondisi MEDIUM :

<!DOCTYPE html>
<html>
    <head>
    <style>
    table {
        border-style: hidden;  /* solid, dotted, dashed, double, groove, ridge, inset, outset, none, hidden */
        }
    </style>
    </head>
    <body>
        <table style="width: 65%;">
        <tr>
            <td>
            <table>
                <tr>
                <th>Metrik</th>
                <th>Kondisi</th>
                <th>Poin Ditambahkan</th>
                </tr>
                    <tr>
                    <!-- # Baris TotaL Cases -->
                    <td> Total Cases per 100rb</td>
                    <td> >= [THRESHOLD_CASES_HIGH]</td>
                    <td span align="center"> 3</td>
                    </tr>
                    <tr>
                    <td></td>
                    <td> >= [THRESHOLD_CASES_MEDIUM]</td>
                    <td span align="center">2</td>
                    </tr>
                    <tr>
                    <td>  </td>
                    <td> > 0</td>
                    <td span align="center"> 1</td>
                    </tr>
                    <tr>
                    <!-- # Baris TotaL Deaths -->
                    <td>Total Deaths per 100rb</td>
                    <td> >= [THRESHOLD_CASES_HIGH]</td>
                    <td span align="center">4</td>
                    </tr>
                    <tr>
                    <td></td>
                    <td> >= [THRESHOLD_CASES_MEDIUM]</td>
                    <td span align="center">2</td>
                    </tr>
                    <tr>
                    <td></td>
                    <td > > 0</td>
                    <td span align="center">1</td>
                    </tr>
                    <tr>
                    <!-- # Baris TotaL Recovered -->
                    <td>Total Recovered per 100rb</td>
                    <td> >= [THRESHOLD_CASES_HIGH]</td>
                    <td span align="center">1</td>
                    </tr>
                    <tr>
                    <td></td>
                    <td> >= [THRESHOLD_CASES_MEDIUM]</td>
                    <td span align="center">0.5</td>
                    </tr>
            </table>
            </td>
            <td>
        <table style="height: 70%;">
                <tr>
                <th>Skor Total</th>
                <th>Zona Risiko</th>
                </tr>
                <tr>
                <td span align="center">>= 5</td>
                <td>Zona Merah</td>
                </tr>
                <tr>
                <td span align="center">>= 2 </td>
                <td>Zona Kuning</td>
                </tr>    
                <tr>
                <td span align="center">< 2</td>
                <td>Zona Hijau</td>
                </tr>    
            </table>
            </td>
        </tr>
        </table>
    </body>
</html>

1. Skor risiko dihitung berdasarkan tiga metrik utama:
   - Total Kasus per 100 ribu penduduk
   - Total Kematian per 100 ribu penduduk dan
   - Total Kesembuhan per 100 ribu penduduk.
1. Setiap metrik memberikan poin berdasarkan ambang batas tertentu:
   - X poin untuk kondisi tinggi dan Y poin untuk kondisi sedang.  
Misalnya, jika Total Kasus per 100 ribu penduduk di atas [threshold A], maka [skor A] ditambahkan.
1. Skor total kemudian digunakan untuk mengklasifikasikan Zona Klasifikasi:
   - Skor >= THRESHOLD High, menjadi Zona Merah
   - Skor <= THRESHOLD High dan >= THRESHOLD Medium, menjadi Zona Kuning
   - Skor < THRESHOLD Medium, menjadi Zona Hijau.

In [ ]:
# --- AMBANG BATAS BERDASARKAN PERSENTIL DATA  ---
# Bisa menggunakan Persentil / Quartile Dalam menentukan THRESHOLD.
# Disi saya menggunakan Persentil ke 90 untuk HIGH dan 50 untuk MEDIUM, yang bertujuan supaya proporsi zona yang berbeda-beda.

# Total Recovered per 100rb
    # Recovered tinggi mungkin bisa berarti penyebarannya luas, jadi kontribusinya ke skor bisa tricky.
    # Jika kita menganggap recovered tinggi sebagai indikator penyebaran awal yang tinggi, maka bisa menambah skor.
    # Jika kita ingin recovered tinggi mengurangi risiko, kita bisa menambahkan logika pengurangan skor.
    # Untuk kesederhanaan awal, kita akan membiarkannya menambah skor jika Anda ingin
    # itu mencerminkan volume epidemi.

def calculate_risk_zone(row):
    score = 0
    # --- Penilaian Skor ---
    # Memberi bobot lebih pada Kasus dan Kematian karena mereka indikator risiko utama
    # Memberi poin lebih tinggi untuk ambang batas yang lebih tinggi

    # Total Cases per 100rb
    # Menggunakan P4 (2744.39) untuk batas "tinggi" dan P2 (678.68) untuk batas "sedang"
    if row["Total Cases per 100rb"] >= 2744.39: # P90
        score += 3
    elif row["Total Cases per 100rb"] >= 678.68: # P50 (Median)
        score += 2
    elif row["Total Cases per 100rb"] > 0: # 0 Artinya terdapat kasus, akan tetapi di bawah rata-rata, maka diberi nilai 1
        score += 1

    # Total Recovered per 100rb (Ini sangat penting untuk risiko)
    # Untuk saat ini recovered tinggi menambah skor, jika itu mencerminkan volume.
    # Namun, seringkali kita ingin recovered tinggi berarti risiko aktif lebih rendah
    # Jika ingin mencerminkan "beban kasus yang sudah sembuh":
    if row["Total Recovered per 100rb"] >= 2669.91: # P90
        score += 1 # Menunjukkan volume kasus yang besar sebelumnya
    elif row["Total Recovered per 100rb"] >= 577.64: # P50 (Median)
        score += 0.5

    # Kesembuhan (Bisa menjadi penambah atau pengurang, tergantung definisi risiko Anda)
    # Jika Anda ingin recovered yang tinggi mengurangi risiko secara keseluruhan:
    # if row["Total Recovered per 100rb"] >= THRESHOLD_RECOVERED_HIGH:
    #     score -= 1 # Contoh: mengurangi 1 poin risiko jika recovered sangat tinggi
    # elif row["Total Recovered per 100rb"] >= THRESHOLD_RECOVERED_MEDIUM:
    #     pass # tidak menambah/mengurangi

    # Total Deaths per 100rb
    # Skalanya lebih kecil dikarenakan persentilnya juga kecil
    if row["Total Deaths per 100rb"] >= 83.47: # P90
        score += 4 # Bobot lebih tinggi karena kematian adalah indikator keparahan
    elif row["Total Deaths per 100rb"] >= 15.81: # P50 (Median)
        score += 2
    elif row["Total Deaths per 100rb"] > 0: # 0 Artinya terdapat Kematian, akan tetapi di bawah rata-rata, maka diberi nilai 1
        score += 1

    # --- Penentuan Zona Akhir ---
    # Tujuannya adalah membagi provinsi ke 3 zona secara lebih merata
    if score >= 5:  # Misalnya, kombinasi tinggi dari dua metrik utama
        return "Zona Merah"
    elif score >= 2: # Kombinasi moderat atau satu metrik tinggi
        return "Zona Kuning"
    else:
        return "Zona Hijau" # kondisi low cases

# Terapkan fungsi ini ke DataFrame Anda
df_zona_risiko["Zona Risiko"] = df_zona_risiko.apply(calculate_risk_zone, axis = 1)

# Setelah menjalankan ini, periksa distribusinya:
print(df_zona_risiko['Zona Risiko'].value_counts())

In [ ]:
df_zona_risiko.head()

In [ ]:
# check apakah nama province di dataset dan json sama
df_zona_risiko['Province'].isin([f['properties']['NAME_1'] for f in geojson_data['features']]).value_counts()

In [ ]:
# check name province in Dataset
print("Nama Provinsi Unik di DataFrame Anda:")
print(sorted(df_zona_risiko['Province'].unique()))

In [ ]:
# check name province in geojson
geojson_province_names = [feature['properties']['NAME_1'] for feature in geojson_data['features']]
print("\nNama Provinsi Unik di GeoJSON Anda:")
print(set(geojson_province_names)) # Menggunakan set untuk nama unik

In [ ]:
# name mapping bertujuan untuk menyamakan Province name di Geojson Dan Dataset.
name_mapping = {'SulawesiBarat': 'Sulawesi Barat',
                'Gorontalo' : 'Gorontalo',
                'MalukuUtara': 'Maluku Utara',
                'Bali' : 'Bali' ,
                'JakartaRaya' : 'DKI Jakarta',
                'Aceh' : 'Aceh',
                'SumateraBarat' : 'Sumatera Barat',
                'JawaTimur' : 'Jawa Timur',
                'BangkaBelitung' : 'Kepulauan Bangka Belitung',
                'JawaBarat' : 'Jawa Barat',
                'PapuaBarat': 'Papua Barat',
                'KalimantanTimur' : 'Kalimantan Timur',
                'SulawesiUtara' : 'Sulawesi Utara',
                'Papua' : 'Papua',
                'SumateraSelatan' : 'Sumatera Selatan',
                'SulawesiTenggara': 'Sulawesi Tenggara',
                'KepulauanRiau' : 'Kepulauan Riau',
                'KalimantanTengah' : 'Kalimantan Tengah',
                'JawaTengah' : 'Jawa Tengah',
                'NusaTenggaraBarat' : 'Nusa Tenggara Barat',
                'KalimantanUtara' : 'Kalimantan Utara',
                'Bengkulu' : 'Bengkulu',
                'Maluku': 'Maluku',
                'NusaTenggaraTimur' : 'Nusa Tenggara Timur',
                'SulawesiTengah' : 'Sulawesi Tengah',
                'Yogyakarta' : 'Daerah Istimewa Yogyakarta',
                'KalimantanBarat' : 'Kalimantan Barat',
                'KalimantanSelatan' : 'Kalimantan Selatan',
                'SulawesiSelatan' : 'Sulawesi Selatan',
                'Jambi' : 'Jambi',
                'SumateraUtara' : 'Sumatera Utara',
                'Riau' : 'Riau',
                'Banten' : 'Banten',
                'Lampung' : 'Lampung'
            }

In [ ]:
# Iterasi melalui setiap fitur (provinsi) di GeoJSON
for feature in geojson_data['features']:
    # Asumsikan nama provinsi ada di feature['properties']['NAME_1']
    current_name = feature['properties'].get('NAME_1') # Gunakan .get() untuk menghindari KeyError

    if current_name in name_mapping:
        feature['properties']['NAME_1'] = name_mapping[current_name]
        print(f"Mengubah '{current_name}' menjadi '{feature['properties']['NAME_1']}'")
    # Opsional: Jika Anda ingin menstandarisasi kapitalisasi atau spasi juga
    # feature['properties']['NAME_1'] = feature['properties']['NAME_1'].lower().strip()

In [ ]:
# check kembali apakah sudah match Province name in Geojson and dataset
df_zona_risiko['Province'].isin([f['properties']['NAME_1'] for f in geojson_data['features']]).value_counts()

In [ ]:
# Membuat peta map sebaran covid-19 di province indonesia
fig = px.choropleth_map(
    df_zona_risiko,
    geojson=geojson_data,
    locations='Province', # Kolom di DataFrame Anda yang berisi nama provinsi (harus unik)
    featureidkey="properties.NAME_1", # Path ke properti di GeoJSON yang berisi nama provinsi yang cocok
    hover_name='Province',
    color='Zona Risiko', # Kolom yang akan menentukan warna (Zona Merah/Kuning/Hijau)
    color_discrete_map={ # Pemetaan warna diskret
        'Zona Merah': 'red',
        'Zona Kuning': 'yellow',
        'Zona Hijau': 'green',
    },
    map_style="carto-positron", # Atau 'open-street-map', dll.
    zoom=4,
    center={"lat": -2.5, "lon": 118}, # Tengah Indonesia
    opacity=0.7,
    # title='Peta Zona Risiko COVID-19 (Choropleth)'
)
fig.update_layout(
    margin={"r":0,"t":30,"l":0,"b":0},
    title={
        'text': "Peta Sebaran Zona Risiko COVID-19 Di indonesia", # Teks judul
        'y': 0.98,              # Posisi vertikal judul (0.0 = bawah, 1.0 = atas figur)
        'x': 0.01,               # Posisi horizontal judul (0.0 = kiri, 0.5 = tengah, 1.0 = kanan figur)
        'xanchor': 'left',    # Anchor horizontal (titik referensi 'x')
        'yanchor': 'top'        # Anchor vertikal (titik referensi 'y')
        }
    )
fig.show()


### **Analisis Peta Zona Risiko Covid-19 di indonesia:**

- Peta diatas menunjukkan sebaran Zona Risiko Covid-19 di Indonesia, dengan 3 parameter yaitu Total Kasus per 100 ribu penduduk, Total Kematian per 100 ribu penduduk dan Total Kesembuhan per 100 ribu penduduk yang di klasifikasikan kedalam 3 Zona Risiko berikut ini :  
    - **Zona Merah (Risiko Tinggi Kasus):** Terlihat terkonsentrasi di beberapa province, terutama di bagian utara dan timur Sumatera, sebagian Kalimantan, Jawa, Bali serta beberapa provinsi di bagian timur Indonesia yaitu papua. Zona Merah ini mengindikasikan tingkat risiko paling tinggi yang ditentukan dari kombinasi kasus, kematian, dan kesembuhan per 100 ribu penduduk. Provinsi-provinsi dengan nilai kematian di atas High Threshold (misalnya, di atas 83.476) kemungkinan besar menjadi faktor utama dalam penentuan "Zona Merah". Wilayah-wilayah ini pada umumnya wilayah padat penduduk dan tingkat urbanisasi tinggi, faktor lainnya mungkin akses ke layanan kesehatan yang belum merata yang mengakibatkan tingginya angka kematian.

    - **Zona Kuning (Risiko Moderat Kasus):** Tesebar luas di beberapa pulau di indonesia, dari Papua, Kalimantan, Sumatera, Sulawesi, Jawa dan beberapa pulau di Nusa Tenggara serta Maluku. Zona Kuning menunjukkan tingkat risiko moderat, yang mengindikasikan perlunya kewaspadaan. Dengan peraturan yang dibuat oleh pemerintah pusat dan pengawalan oleh pemerintah daerah, membuat transmisi epidemi menjadi terkontrol.  

    - **Zona Hijau (Risiko Rendah Kasus):** Terlihat di beberapa province, hal ini menandakan keberhasilan dalam mengatasi pandemi dengan lebih efektif. Seperti sebagian besar Pulau Sulawesi dan Nusa Tenggara Timur, beberapa wilayah di Sumatera, dan sebagian kecil di Kalimantan, Jawa dan Maluku. Zona Hijau menunjukkan tingkat risiko terendah, kemungkinan disebabkan karena jumlah kasus, kematian, dan kesembuhan yang cenderung rendah per 100 ribu penduduk.  

Interpretasi Awal: Pola ini sangat mungkin mencerminkan pusat-pusat kepadatan penduduk, mobilitas ekonomi, dan interaksi sosial yang tinggi. Provinsi-provinsi dengan population yang tinggi, contohnya di pulau Jawa sebagai pusat ekonomi dan populasi, secara alami akan memiliki jumlah kasus kumulatif yang lebih tinggi. Dan normalisasi 'per 100rb' mengonfirmasi bahwa penularan juga signifikan di pulau Jawa, yang relatif terhadap jumlah penduduknya.

## **Provinces with the highest number of cases**

In [ ]:
# Copy dataframe yang sudah cleaned
df_province = province_stats_filtered.copy()

In [ ]:
df_province.info()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.ticker as ticker
from ipywidgets import VBox, Label
# 1. Kolom yang akan divisualisasikan
cases_col = ['Cases', 'Deaths', 'Recovered']
cases_total = ['Total Cases', 'Total Deaths', 'Total Recovered']

# 2. Filter provinsi saja
prov_df = df_province[df_province['Location Level'] == 'Province']

# 3. Ambil data terakhir per provinsi (berdasarkan tanggal terakhir tersedia per provinsi)
prov_latest = prov_df.groupby('Location').last()

# 4. Membuat fungsi untuk plot berdasarkan pilihan
def plot_top10(column):

    clear_output(wait=True)  # Untuk menghapus output sebelumnya
    display(dropdown)        # Untuk menampilkan ulang dropdown setelah clear

    # Ambil 10 provinsi dengan nilai tertinggi untuk kolom tersebut
    top10 = prov_latest.sort_values(by=column, ascending=False).head(10)

    # Plotting
    plt.figure(figsize=(17, 8))
    ax = plt.gca()
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))  # Untuk mengubah format angka Y
    bars = plt.bar(top10.index, top10[column], color='cornflowerblue')
    plt.title(f'10 Provinces with the Highest {column}')
    plt.xlabel('Province')
    plt.ylabel(col)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y')

    # Tambahkan label nilai
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + yval*0.02, f'{int(yval):,}', ha='center', va='bottom', fontsize=9) #supported values are 'top', 'bottom', 'center', 'baseline', 'center_baseline'

    # Untuk mengatur padding pada grafik
    plt.tight_layout()

    # Untuk menampilkan Grafik
    plt.show()

# Dropdown widget
dropdown = widgets.Dropdown(
    options =['Total Cases', 'Total Deaths', 'Total Recovered'],
    value = 'Total Cases',
    description='Choose Case: ',
    layout=widgets.Layout(width='250px')  # optional
)

# Ubah posisi description ke atas
dropdown.style.description_width = 'initial'

# Hubungkan dropdown ke fungsi
dropdown.observe(lambda change: plot_top10(change['new']), names='value')

# Tampilkan pertama kali
plot_top10(dropdown.value)